In [1]:
import pandas as pd
jurnal = pd.read_csv(r"C:\Users\Fauzan Anshori\Downloads\Jurnal ML\datajurnal2.csv",sep=";")

In [2]:
jurnal.head()
#jurnal["similarity"]

,journalId,title,abstract,keywords
0,1,Mesenchymal Transformation: The Rosetta Stone ...,"Despite decades of research, glioblastoma (GB...",clinical outcome|glioblastoma (GBM)|mesenchyma...
1,2,"DGAT1 protects tumor from lipotoxicity, emergi...","We recently demonstrated that glioblastoma, th...",DGAT1|glioblastoma|lipotoxicity|oxidative stre...
2,3,Anti-PD-1 Immunotherapy in Preclinical GL261 G...,Glioblastomas (GBs) are malignant brain tumou...,TMZ|checkpoint inhibitors|glioblastoma|long-te...
3,4,Recent Updates on the Relationship between Can...,Autoimmune pancreatitis (AIP) is now considere...,IgG4-related disease|autoimmune pancreatitis|c...
4,5,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...,cancer|cure.


In [3]:
import nltk
import nltk.tokenize


# Tokenizing
nltk.download('punkt')
# Stopwords removal
nltk.download('stopwords')
# Lemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

print('Completed')

Completed


[nltk_data] Downloading package punkt to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
import re

def clean_title(title):
  re.sub("[^a-zA-Z0-9 ]", "", title)
  return title

In [5]:
jurnal["clean_title"] = jurnal["title"].apply(clean_title)

In [7]:
jurnal
print(jurnal.shape)

(90, 5)


In [6]:
import re

def clean_abstract(abstract):
  re.sub("[a-zA-Z ]", "", abstract)
  return abstract

In [7]:
jurnal["clean_abstract"] = jurnal["abstract"].apply(clean_abstract)
print(jurnal.shape)

(90, 6)


In [31]:
# jurnal['word_count'] = jurnal['abstract'].apply(lambda x: len(x.split()))
# #jurnal.groupby('keywords')['word_count'].mean()
# jurnal.groupby('abstract')['word_count'].mean()
# jurnal

**Vectorizer**

In [10]:
from numpy.lib.function_base import vectorize
from sklearn.feature_extraction.text import TfidfVectorizer #mengubah kata menjadi angka

vectorizer = TfidfVectorizer(ngram_range=(1, 2))

tfidf = vectorizer.fit_transform(jurnal["clean_title"])
print(jurnal.shape)

(90, 6)


**1. similarity berdasarkan title**

In [43]:
from sklearn.metrics.pairwise import cosine_similarity # menggunakan pairwise untuk menghitung kedekatan antar kata
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

cv=CountVectorizer(max_features=90,stop_words='english')
print(cv)
vector=cv.fit_transform(jurnal['abstract'].values.astype('U')).toarray()
print(vector.shape)
similar= cosine_similarity(vector)


# def search(title): # Membuat fungsi search
#     #print(title)
#     title = clean_title(title) # Memanggil fungsi clean_title
#     #print(title)
#     query_vec = vectorizer.transform([title]) # Mengubah isi dataframe title menjadi vektor
#     #print(query_vec.shape)
#     #print(tfidf.shape)
#     similarity = cosine_similarity(query_vec, tfidf).flatten() # Perhitungan cosinus antara pencarian kata dengan dataset
#     indices = np.argpartition(similarity, -5)[-5:] # Mendapatkan 5 data terdekat dari hasil pencarian
#     results = jurnal.iloc[indices].iloc[::-1] # Menunjukan lokasi data bedasarkan indices

#     print(similarity[indices])
    
#     return results

# search("Internet of thing")

CountVectorizer(max_features=90, stop_words='english')
(90, 90)


In [34]:
# Membuat search bar interaktif
import ipywidgets as widgets
from IPython.display import display

jurnal_input = widgets.Text( # Variabel input search bar
    value='Cancer and cure: A critical analysis',
    description='jurnal Title:',
    disabled=False
)
jurnal_list = widgets.Output() # Menampilkan search bar

def on_type(data): # Mencari data pada setiap ketikan input
    with jurnal_list:
            jurnal_list.clear_output()
            title = data["new"]
            if len(title) > 5: # Membatasi minimal 5 karakter input untuk search
                display(search(title)) # Menampilkan hasil search kepada output widget

jurnal_input.observe(on_type, names='value') # Memanggil fungsi on_type ketika terjadi perubahan pada input bar

display(jurnal_input, jurnal_list) # Menampilkan kedua widget yang dibuat

Text(value='Cancer and cure: A critical analysis', description='jurnal Title:')

Output()

**2. Similarity berdasarkan Abstract**

In [12]:
from sklearn.metrics.pairwise import cosine_similarity # menggunakan pairwise untuk menghitung kedekatan antar kata
import numpy as np

print(vectorizer)

def search(abstract): # Membuat fungsi search
    print(abstract)
    abstract = clean_abstract(abstract) # Memanggil fungsi clean_abstract
    query_vec = vectorizer.transform([abstract]) # Mengubah isi dataframe abstract menjadi vektor
    print(query_vec)
    similarity = cosine_similarity(query_vec, tfidf).flatten() # Perhitungan cosinus antara pencarian kata dengan dataset
    indices = np.argpartition(similarity, -5)[-5:] # Menampilkan data terdekat dari hasil pencarian, mengurutkan nilai besar ke paling depan
    results = jurnal.iloc[indices].iloc[::1] # Menunjukan lokasi data bedasarkan indices
    
    #jurnal['similarity'] = jurnal['abstract'].apply(similarity)
    #jurnal.groupby('clean_abstract')['similarity'].mean()
    #rec_percentages.head(10).merge(jurnal, left_index=True, right_on="title")
    #print(similarity)

    print(similarity[indices])
    return results

search("Cancer and cure: A critical analysis")

 

TfidfVectorizer(ngram_range=(1, 2))
Cancer and cure: A critical analysis
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (1, 1110)>
  Coords	Values
  (0, 48)	0.32947333218202296
  (0, 51)	0.1609079318976815
  (0, 57)	0.3848424446147665
  (0, 162)	0.2009102344354586
  (0, 164)	0.35245359014339805
  (0, 247)	0.32947333218202296
  (0, 248)	0.3848424446147665
  (0, 251)	0.3848424446147665
  (0, 252)	0.3848424446147665
[0.06390462 0.08281529 0.12465054 1.         0.09955284]


,journalId,title,abstract,keywords,clean_title,clean_abstract
11,12,Detection of Cancer DNA in Early Stage and Met...,Breast cancer is the leading cause of cancer i...,Biopsy|Blood|Breast cancer|Droplet digital|Li...,Detection of Cancer DNA in Early Stage and Met...,Breast cancer is the leading cause of cancer i...
35,36,Machine learning for internet of things data a...,"Rapid developments in hardware, software, and ...",internet of things.,Machine learning for internet of things data a...,"Rapid developments in hardware, software, and ..."
3,4,Recent Updates on the Relationship between Can...,Autoimmune pancreatitis (AIP) is now considere...,IgG4-related disease|autoimmune pancreatitis|c...,Recent Updates on the Relationship between Can...,Autoimmune pancreatitis (AIP) is now considere...
4,5,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...,cancer|cure.,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...
85,86,Exact Controllability and Asymptotic Analysis ...,the authors consider the exact controllability...,NaN,Exact Controllability and Asymptotic Analysis ...,the authors consider the exact controllability...


In [36]:
#import streamlit as st
#import pickle
#import requests

#st.title('Sistem Rekomendasi Jurnal')
#selected_journal_name = st.selectbox(
##    (["Measuring cancer evolution from the genome"]))

In [37]:
# def add_bg_from_url():
#     st.markdown(
#          f"""
#          <style>
#          .stApp {{
#              background-image: url("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR60O2-muWvwgWvwmzklx-IydP5j0NZsLnHpg&usqp=CAU");
#              background-attachment: fixed;
#              background-size: cover
#          }}
#          </style>
#          """,
#          unsafe_allow_html=True
#      )

In [9]:
import pickle

In [10]:
pickle.dump(jurnal, open('jurnal_list.pkl', 'wb'))

In [40]:
pickle.dump(vector, open('similarity.pkl', 'wb'))
print(vector.shape)

(90, 90)


In [41]:
pickle.load(open('jurnal_list.pkl', 'rb'))


,journalId,title,abstract,keywords,clean_title,clean_abstract
0,1,Mesenchymal Transformation: The Rosetta Stone ...,"Despite decades of research, glioblastoma (GB...",clinical outcome|glioblastoma (GBM)|mesenchyma...,Mesenchymal Transformation: The Rosetta Stone ...,"Despite decades of research, glioblastoma (GB..."
1,2,"DGAT1 protects tumor from lipotoxicity, emergi...","We recently demonstrated that glioblastoma, th...",DGAT1|glioblastoma|lipotoxicity|oxidative stre...,"DGAT1 protects tumor from lipotoxicity, emergi...","We recently demonstrated that glioblastoma, th..."
2,3,Anti-PD-1 Immunotherapy in Preclinical GL261 G...,Glioblastomas (GBs) are malignant brain tumou...,TMZ|checkpoint inhibitors|glioblastoma|long-te...,Anti-PD-1 Immunotherapy in Preclinical GL261 G...,Glioblastomas (GBs) are malignant brain tumou...
3,4,Recent Updates on the Relationship between Can...,Autoimmune pancreatitis (AIP) is now considere...,IgG4-related disease|autoimmune pancreatitis|c...,Recent Updates on the Relationship between Can...,Autoimmune pancreatitis (AIP) is now considere...
4,5,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...,cancer|cure.,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...
...,...,...,...,...,...,...
85,86,Exact Controllability and Asymptotic Analysis ...,the authors consider the exact controllability...,NaN,Exact Controllability and Asymptotic Analysis ...,the authors consider the exact controllability...
86,87,Existence and Asymptotic Behavior of Radially ...,This paper is concerned with a class of semili...,NaN,Existence and Asymptotic Behavior of Radially ...,This paper is concerned with a class of semili...
87,88,Global Bifurcation of a Perturbed Double-Homoc...,This paper deals with a kind of fourth degree ...,NaN,Global Bifurcation of a Perturbed Double-Homoc...,This paper deals with a kind of fourth degree ...
88,89,Global Existence of Classical Solutions for So...,"In this paper, we prove local and global exist...",NaN,Global Existence of Classical Solutions for So...,"In this paper, we prove local and global exist..."
